In [2]:
import openai
import pandas as pd
import seaborn as sns
import time
import os

# Set OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [14]:
## access the posts csvs
post_list_1 = pd.read_csv(r'data\april_2024\accepted_post_list_2.csv').Text.tolist()
post_list_2 = pd.read_csv(r'data\april_2024\approved_post_list_2.csv').Text.tolist()
post_list_3 = pd.read_csv(r'data\april_2024\refused_post_list_2.csv').Text.tolist()
post_list_4 = pd.read_csv(r'data\april_2024\rejected_post_list_2.csv').Text.tolist()
post_list = post_list_1 + post_list_2 + post_list_3 + post_list_4

# write csv

In [98]:
post_list = pd.read_csv(r'data\all_posts\all_posts_agg.csv')
post_list.drop_duplicates(inplace=True)
post_list = post_list["0"].tolist()

In [99]:
### remove posts that are too short or too long
for post in post_list:
        if len(post) < 300 or len(post) > 6000:
            post_list.remove(post)

In [100]:
import random

random.shuffle(post_list)

In [135]:
from openai import OpenAI
client = OpenAI()
messages = []
responses_5 = []
disclaimer = 'One-two words answers only. Answer in the JSON format.{\n   "Interview Location": "Islamabad/Karachi",\n  "Program": "Data Science",\n  "Degree Level": "Masters",\n  "University": "Unversity of California Davis",\n  "Universities Applied": "3",\n  "Sponsor": "Uncle",\n  "Scholarship": "No",\n  "Relatives in US": "No",\n  "Visa Attempts": "First",\n  "Visa Result": "Refused"}'
system_message = "You are a keyword extractor! Extract key information from the narrative and present it in JSON format."
instructions = "\n\n\nInclude the following fields: Interview Location(Islamabad/Karachi//N/A), Program, Degree Level(Community College, Bachelors, Masters, Phd), University(Give full name of university such as: SDSU -> San Diego State University), Universities Applied(1/2/3/...), Sponsor(Father/Parents/Uncle/Self/Employer/University/...), Scholarship (Yes/Partial/No//N/A), Relatives in US (Parents/Siblings/Relatives(Uncle/Aunt/...)/ ...), Visa Attempts(First/Second/Third/Fourth), and Visa Result(Approved/Refused). Use 'N/A' for not given fields. Provide full name of university even it is not given in the story like UCD should be University of California Davis.\n\n\n."
model_agreement = "Ok. I understand that you want to extract keyword information from the story. Please give me a story."
sample_question = "Visa interview Islamabad Texas A&M university 5th Aug: 11:30am Vo: Hello Me:hello sir  Vo: i need u to do your finger prints. (Messed up with left hand once) Me:done Vo: alright pass me those documents. Me: here Vo: so why are you going to the Us? Me: I'm going to pursue my bachelor's degree in health sciences at texas a&m university. Vo: how many universities did you apply to? Me: about 13 universities, I got accepted from a few. Vo: and which ones did u get accepted from? Me: i got accepted from the university of Hartford, texas a&m and university of---uh and I had a correspondence with the Baylor university. Vo: ok, so why did you choose this university? Me: well it has quite good ranking regarding my course, its listed as the 28th best medical university in terms of health care management and in biological scien--- Vo: alright so do you have any relatives in the Us? Me: Yes, an uncle of mine lives there with his family. Vo: where do they live? Me: Irving (accident said the old address) Vo: California?-- what state? Me: no not California, its in Dallas at Texas. Vo: alright and what does he do? Me: his work is associated with transport but I don't really know much about that. Vo: oh okay, so whose going to sponsor you? Me: my father will Vo: and what does he do? Me: told Vo: (going though my docs) have you ever been to the Us before? Me: yes, twice Vo: okay. (Keeps typing and looking through stuff for about a minute or two). Vo: alright, I'm approving your visa, but we'll be keeping your passport for further processing. You will receive it within 1-2weeks. There were some 30 candidates I saw that day and out of them only 2 girls & me myself got approved. One common thing between me and one of those girls was that we both had been to the Us before. Me twice for visiting and her for her bachelors(now going for masters I believe."
sample_answer = '{\n  "Interview Location": "Islamabad",\n  "Program": "Health Sciences",\n  Degree Level": "Bachelors",\n  "University": "Texas A&M University",\n  "Universities Applied": 13,\n  "Sponsor": "Father",\n  "Scholarship": "N/A",\n  "Relatives in US": "Uncle",\n  "Visa Attempts": "First",\n  "Visa Result": "Approved"}'

for i in range(797, len(post_list)):
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "system", "content": system_message},
                {"role": "user", "content": disclaimer},
                {"role": "assistant", "content": model_agreement},
                {"role": "user", "content": instructions + sample_question},
                {"role": "assistant", "content": sample_answer},
               {"role": "user", "content": post_list[i]}],
        temperature = 0
    )
    responses_5.append(completion.choices[0].message.content)

In [136]:
responses_list = responses + responses_2 + responses_3 + responses_4 + responses_5

In [137]:
print(f"The number of data we have is : {len(responses_list)}")

The number of data we have is : 879


In [138]:
import json
# Convert each JSON string to a dictionary
json_responses = [json.loads(s) for s in responses_list]


In [139]:
df = pd.DataFrame(json_responses)
df = df[["Interview Location", "Program", "Degree Level", "University", "Universities Applied", "Sponsor", "Scholarship", "Relatives in US", "Visa Attempts", "Visa Result"]]
df

,Interview Location,Program,Degree Level,University,Universities Applied,Sponsor,Scholarship,Relatives in US,Visa Attempts,Visa Result
0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Approved
1,Islamabad,Computer Science,Bachelors,University of Illinois at Chicago,3,Father,N/A,N/A,First,Refused
2,N/A,Psychology,Bachelors,Kansas State University,N/A,N/A,N/A,N/A,N/A,N/A
3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,First,Approved
4,Islamabad,Computer Science,Masters,George Washington University,N/A,Father/Uncle,"$37,000",Uncle,N/A,Approved
...,...,...,...,...,...,...,...,...,...,...
874,Karachi,Health and Hygiene Conference,N/A,N/A,N/A,Self,N/A,Husband,First,Approved
875,Islamabad,Agriculture,Bachelors,Southwest Minnesota State University,1,Father,N/A,N/A,Second,Approved
876,Islamabad,Computer Science,Bachelors,Tennessee Tech University,7,Father,N/A,N/A,First,Refused
877,N/A,Bachelor of Accounting,Bachelors,Kent State University,Multiple,N/A,N/A,N/A,Second,N/A


In [140]:
### Saving the data
df.to_csv(r"data\training_data_april_2024.csv")